<a href="https://colab.research.google.com/github/Abhishek3102/GraphRAG/blob/main/GraphRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-experimental langchain-community langchain networkx langchain-google-genai langchain-core json-repair

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18


In [7]:
import os
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_genai import GoogleGenerativeAI
import time
import networkx as nx
from langchain.chains import GraphQAChain
from langchain_core.documents import Document
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
import google.generativeai as genai

In [ ]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

In [11]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [30]:
# llm = GoogleGenerativeAI(model="gemini-pro", google_api_key = GOOGLE_API_KEY)
llm = GoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", google_api_key = GOOGLE_API_KEY)
text = """
Eric Cantona, born in France in 1966, is considered one of football's most iconic and charismatic players. He began his career in France before making a legendary move to England, joining Leeds United in 1992, and then Manchester United. Known for his bold personality and creativity, Cantona played a pivotal role in United's dominance during the 1990s. He won four Premier League titles and two FA Cups with the club, becoming a symbol of success.

Cantona was known for his dazzling skills, unpredictable behavior, and iconic moments, including his infamous "kung fu" kick against a fan in 1995. Despite his often controversial actions, he was admired for his leadership and ability to turn games with his flair and vision. After retiring from football in 1997, Cantona ventured into acting, further cementing his status as a cultural figure. His legacy remains integral to Manchester United's history.
"""

In [31]:
documents = [Document(page_content=text)]
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [32]:
llm_transformer_filtered = LLMGraphTransformer(llm=llm, allowed_nodes=["Person", "Country", "Organization"],
allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "PLAYING_STYLE"],)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(documents)

In [33]:
graph = NetworkxEntityGraph()

# Adding Nodes to the Graph
for node in graph_documents_filtered[0].nodes:
  graph.add_node(node.id)

# Adding Edges to the Graph
for edge in graph_documents_filtered[0].relationships:
  graph._graph.add_edge(edge.source.id, edge.target.id, relation=edge.type,)

In [34]:
chain = GraphQAChain.from_llm(llm=llm, graph=graph, verbose=True)

In [35]:
question = """ How was Eric Cantona?"""
chain.run(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Eric Cantona
Full Context:
Eric Cantona NATIONALITY France
Eric Cantona WORKED_AT Leeds United
Eric Cantona WORKED_AT Manchester United

> Finished chain.


"I don't know"